In [ ]:
# First, parse our California school data from a CSV bundled in the repo
import csv,codecs

i = lambda x: int(x.replace(',','') or 0) # Quick function to make "123,123" into an int 123123
cupc_csv_file = 'working/jupyter.csv'
schools_data = [
    r for r in 
    csv.DictReader(codecs.open(cupc_csv_file)) 
    if i(r['total_enrolled']) > 0 # ignore 0 student schools
] 

schools_data

In [ ]:
# The CEPSchool and CEPDistrict classes encapsulate our data model
# We first take the raw school data and push it into CEPDistrict and CEPSchools,
# reducing it to a list of Districts, each with its list of schools
from strategies.base import CEPSchool, CEPDistrict
districts = {}
for row in schools_data:
    school = CEPSchool(row)
    if row['district'] not in districts:
        district = CEPDistrict(row['district'],row['district'])
        districts[row['district']] = district
    districts[row['district']].add_school(school)
districts = list(districts.values())
districts.sort()
districts

In [ ]:
# The CEPDistrict class represents a School District
# it has a list of CEPSchools, and provides some aggregate information

# Let's inspect a district
district = districts[0]
district.name,len(district.schools),district.code,district.overall_isp,district.total_enrolled

    

In [ ]:
# We can look at all the schools
from IPython.display import display, HTML
import tabulate

school_table = [('School','Total Enrolled','ISP%','Breakfast','Lunch')] + \
            [ (s.name,s.total_enrolled,('%0.2f%%' % (s.isp*100.0)),s.bfast_served,s.lunch_served, s.active ) 
              for s in district.schools ]

display(HTML(tabulate.tabulate(school_table,tablefmt='html')))


In [ ]:
# Or do some charts
import matplotlib.pyplot as plt

plt.hist([s.total_enrolled for s in district.schools], 20, label='Label')
plt.show()

In [ ]:
# Strategies are the algorithms we use to group schools
# Each one extends BaseStrategy and overrides the "create_groups" method
# Here we need to define which strategies we want to run
# We pass them into parse_districts as the class name, any parameters for it, and a name
# this way we can run the sam strategy in multiple configurations

from strategies.naive import OneGroupCEPStrategy,OneToOneCEPStrategy

# If we create a strategy, and run its create_groups method with the district as the
# incoming parameter, then it fills its own "groups" member with the resulting CEPGroup objects
#strategy = OneGroupCEPStrategy()
strategy = OneToOneCEPStrategy()
strategy.create_groups(district)

# In the case of OneGroup, we have only 1 group in our list
g = strategy.groups[0]
g,g.covered_students,g.est_reimbursement()
strategy.reimbursement * 180

In [ ]:
# We can see the recommended grouping of schools for the strategy result
from strategies.pairs import PairsCEPStrategy
strategy = PairsCEPStrategy()

strategy.create_groups(district)

school_groupings = [('Group','School','Group ISP','School ISP','School Type','School Total Enrolled')]
for g in strategy.groups:
    for s in g.schools:
        school_groupings.append( (g.name,s.name,g.isp,s.isp,s.school_type,s.total_enrolled))
display(HTML(tabulate.tabulate(school_groupings,tablefmt='html')))


In [ ]:
from strategies.nyc_moda_simulated_annealing import NYCMODASimulatedAnnealingCEPStrategy
strategy = NYCMODASimulatedAnnealingCEPStrategy({"fresh_starts":50,"iterations":1000})
groups = strategy.create_groups(district)
# Baseline from what mealscount.com does
strategy.reimbursement * 180

In [ ]:
# no demonstrable increase
#strategy = NYCMODASimulatedAnnealingCEPStrategy({"fresh_starts":500,"iterations":5000})
#groups = strategy.create_groups(district)
#Increased iteration, Best Groupings Assuming Avg Count
#strategy.reimbursement * 180

In [ ]:
# Custom for Oakland Unified
# Build list of school data
school_participations = {}
for s in schools_data:
    school_participations[s['school_code']] = {
        'bfast':int(s['daily_breakfast_served']),
        'lunch':int(s['daily_lunch_served']),
        'free':float(s['% free'].rstrip('%'))/100,
        'reduced':float(s['% reduced'].rstrip('%'))/100,
    }
school_participations[0]

In [ ]:
for s in district.schools:
    sdata = school_participations[s.code]
    s.bfast_served = round(s.total_enrolled * (sdata['free']+sdata['reduced']))
    s.lunch_served = round(s.total_enrolled * (sdata['free']+sdata['reduced']))
strategy = NYCMODASimulatedAnnealingCEPStrategy({"fresh_starts":50,"iterations":1000})
groups = strategy.create_groups(district)
#Increased iteration, Best Groupings, frpm students eat bfast/lunch
strategy.reimbursement * 180

In [ ]:
for s in district.schools:
    sdata = school_participations[s.code]
    s.bfast_served = s.total_enrolled
    s.lunch_served = s.total_enrolled
strategy = NYCMODASimulatedAnnealingCEPStrategy({"fresh_starts":50,"iterations":1000})
groups = strategy.create_groups(district)
#Increased iteration, Best Groupings, all students eat bfast/lunch
strategy.reimbursement * 180

In [ ]:
# IMPORTANT: Reset data
for s in district.schools:
    sdata = school_participations[s.code]
    s.bfast_served = sdata['bfast']
    s.lunch_served = sdata['lunch']